In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing as mp
import os
os.chdir("The_single_cell_transcriptome_as_a_component_system/MouseCellAtlas/")

# all data pdf

In [ ]:
n_bins = 50
bins = np.logspace(0,2, n_bins)
def get_tissue_bc(tissue, bins=bins):
    df = pd.read_csv(f"mca/mainTable_{tissue}.csv", index_col=0, header=0)
    #f = df.divide(df.sum(0),1).mean(1).values
    #f = f/f.sum()
    all_data = df.values
    all_data = all_data[all_data>0]
    bc = np.histogram(all_data, bins=bins)[0]
    return bc

In [ ]:
pool = mp.Pool(12)

arr = ["Brain", "Liver", "Muscle", "Bladder", "Prostate", "Pancreas", "Stomach", "Kidney", "Ovary", "Uterus", "Lung"]
w = pool.map_async(get_tissue_bc, arr, error_callback=lambda err:print(err))

pool.close()
pool.join()
bin_counts = np.mean(w.get(), axis=0)

In [ ]:
plt.figure(figsize=(9,8))

plt.hlines(bin_counts, bins[:-1],bins[1:], lw=10, color="gray")

plt.xlabel("f")
plt.ylabel("pdf")
#plt.yscale("log")
plt.xscale("log")

In [ ]:
df = pd.read_csv(f"mca/mainTable_Kidney.csv", index_col=0, header=0)

In [ ]:
M = df.sum(0)
f = df.divide(df.sum(0),1).mean(1)
f = f/f.sum()
M_tilde = 25000

In [ ]:
all_data = df.values
all_data = all_data[all_data>0]

In [ ]:
plt.figure(figsize=(9,8))

plt.hist(all_data, bins=np.linspace(1,100,100), lw=10, color="gray", density=True)
plt.scatter(1,1)

plt.xlabel("f")
plt.xlabel("pdf")
plt.yscale("log")

In [ ]:
X = [fi*M*np.exp(-M/M_tilde) for fi in f]

In [ ]:
P1 = pd.DataFrame(data=[(np.exp(fi*M)*(M/M_tilde)*x*np.exp(x)).values for fi,x in zip(f,X)], index=f.index, columns=M.index)

In [ ]:
P10 =[x*M/M_tilde for x in X]

In [ ]:
np.mean(P10)*0.95